## Logistic Regression

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

if len(x_data) != len(y_data):
    print("data_error")

In [3]:
X = tf.placeholder(tf.float32, shape = [None,len(x_data[0])])
Y = tf.placeholder(tf.float32, shape = [None,len(y_data[0])])

In [4]:
W = tf.Variable(tf.random_normal([2,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

In [5]:
hypothesis = tf.sigmoid(tf.matmul(X,W) + b)

In [6]:
cost = - tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [7]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost,train], feed_dict = {X:x_data, Y: y_data})
        if step% 1000 == 0:
            print(step,cost_val)
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                  feed_dict = {X:x_data, Y: y_data})

    print(h,"\n", c,"\n",a)

0 1.90236
1000 0.362486
2000 0.310481
3000 0.270755
4000 0.239308
5000 0.214023
6000 0.193384
7000 0.17629
8000 0.161938
9000 0.14974
10000 0.139256
[[ 0.02655357]
 [ 0.152753  ]
 [ 0.28416067]
 [ 0.7910077 ]
 [ 0.94547874]
 [ 0.98216808]] 
 [[ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]] 
 1.0


## Real Data 

In [9]:
xy = np.loadtxt('DeepLearningZeroToAll/data-03-diabetes.csv', delimiter=',',dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
print(x_data.shape, y_data.shape)

(759, 8) (759, 1)


In [10]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 1.21915
200 0.696728
400 0.609692
600 0.589769
800 0.579867
1000 0.572214
1200 0.565551
1400 0.559599
1600 0.554243
1800 0.549405
2000 0.54502
2200 0.541035
2400 0.537401
2600 0.534079
2800 0.531033
3000 0.528233
3200 0.525652
3400 0.523266
3600 0.521056
3800 0.519005
4000 0.517095
4200 0.515314
4400 0.51365
4600 0.512092
4800 0.51063
5000 0.509256
5200 0.507962
5400 0.506742
5600 0.50559
5800 0.504501
6000 0.503469
6200 0.50249
6400 0.50156
6600 0.500676
6800 0.499834
7000 0.499031
7200 0.498266
7400 0.497535
7600 0.496835
7800 0.496166
8000 0.495525
8200 0.49491
8400 0.494321
8600 0.493754
8800 0.49321
9000 0.492686
9200 0.492182
9400 0.491696
9600 0.491228
9800 0.490777
10000 0.490341

Hypothesis:  [[ 0.46312627]
 [ 0.94117445]
 [ 0.12315116]
 [ 0.95213789]
 [ 0.28633043]
 [ 0.6806221 ]
 [ 0.94299603]
 [ 0.59210849]
 [ 0.20398466]
 [ 0.51741683]
 [ 0.70045578]
 [ 0.1503302 ]
 [ 0.24915262]
 [ 0.41841549]
 [ 0.73636007]
 [ 0.49945021]
 [ 0.6931892 ]
 [ 0.89563471]
 [ 0.81950122]
 [

## Real data with batch

In [12]:
filename_queue = tf.train.string_input_producer(
    ['DeepLearningZeroToAll/data-03-diabetes.csv'],shuffle = False, name ="filename_queue")

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [tf.constant([], dtype=tf.float32) for i in range(8)] + [tf.constant([], dtype=tf.int32)]
xy = tf.decode_csv(value, record_defaults = record_defaults, field_delim=',')

In [13]:
train_x_batch , train_y_batch = \
    tf.train.batch([xy[0:-1],xy[-1:]],batch_size = 10)

In [14]:
train_x_batch.shape[1]

Dimension(8)

In [15]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [17]:
with tf.Session() as sess:
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    
    for step in range(10001):
        x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_batch, Y: y_batch})
        if step % 500 == 0:
            print(step, cost_val)


    coord.request_stop()
    coord.join(threads)
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})

0 0.623662
500 0.375897
1000 0.418574
1500 0.458226
2000 0.437303
2500 0.626499
3000 0.519785
3500 0.368157
4000 0.613791
4500 0.635387
5000 0.478897
5500 0.511526
6000 0.447231
6500 0.433681
7000 0.296365
7500 0.446585
8000 0.381242
8500 0.343454
9000 0.531425
9500 0.530036
10000 0.338958


In [18]:
print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)


Hypothesis:  [[ 0.41221207]
 [ 0.92515236]
 [ 0.2478793 ]
 [ 0.94886899]
 [ 0.17639628]
 [ 0.76320016]
 [ 0.9500525 ]
 [ 0.62515843]
 [ 0.28108796]
 [ 0.4950245 ]
 [ 0.6647954 ]
 [ 0.17496838]
 [ 0.2312711 ]
 [ 0.30748188]
 [ 0.75271958]
 [ 0.42317981]
 [ 0.7300418 ]
 [ 0.89672971]
 [ 0.81732923]
 [ 0.5402903 ]
 [ 0.64574188]
 [ 0.09840988]
 [ 0.64241523]
 [ 0.69828081]
 [ 0.36737153]
 [ 0.93640834]
 [ 0.57453477]
 [ 0.60340333]
 [ 0.69043326]
 [ 0.42597187]
 [ 0.95702356]
 [ 0.85363275]
 [ 0.57512534]
 [ 0.8306371 ]
 [ 0.36568692]
 [ 0.62682021]
 [ 0.81369269]
 [ 0.52100772]
 [ 0.49768117]
 [ 0.33285177]
 [ 0.81000745]
 [ 0.11843222]
 [ 0.44560868]
 [ 0.05126576]
 [ 0.61257774]
 [ 0.92399412]
 [ 0.70761615]
 [ 0.72911948]
 [ 0.93279815]
 [ 0.94318551]
 [ 0.93627059]
 [ 0.22605208]
 [ 0.36789185]
 [ 0.96985435]
 [ 0.23285864]
 [ 0.42549226]
 [ 0.1032305 ]
 [ 0.72731346]
 [ 0.88168532]
 [ 0.5157423 ]
 [ 0.94848114]
 [ 0.73794448]
 [ 0.67958802]
 [ 0.84699935]
 [ 0.53976357]
 [ 0.516628